# Fine-mapping of PD-related risk loci in East Asian summary statistics
* Project: Cross-ancestry PAR
* Version: R/4.4
* Status: Complete
* Last Updated: 13-FEB-2025

## Notebook overview
* Extract chromosome and base pair positions from summary statistics for selected loci
* Perform fine-mapping and save results

In [1]:
library("data.table")
#if (!requireNamespace("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")
#BiocManager::install("snpStats")
library("robustbase")
library(ggplot2)
library(tidyr)
devtools::install_github("chr1swallace/coloc")
library("coloc")
library("tidyverse")
library("readr")

Skipping install of 'coloc' from a github remote, the SHA1 (fd1c0351) has not changed since last install.
  Use `force = TRUE` to force installation

This is coloc version 5.2.3

── Attaching core tidyverse packages ────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ lubridate 1.9.4     ✔ tibble    3.2.1
✔ purrr     1.0.2     
── Conflicts ──────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()     masks data.table::between()
✖ dplyr::filter()      masks stats::filter()
✖ dplyr::first()       masks data.table::first()
✖ lubridate::hour()    masks data.table::hour()
✖ lubridate::isoweek() masks data.table::isoweek()
✖ dplyr::lag()         masks stats::lag()
✖ dplyr::last()        masks data.table::last()
✖ lubridate::mday()    masks data.table::mday()
✖ lubridate::minute()  masks data.table::minute()
✖ lubridate::month()   masks data.table::month()
✖ lubri

In [2]:
## Read dataframe
df0 <- fread("{WORK_DIR}/Sumstat/per-cohort/EAS.final.txt", header =T)
head(df0)

MARKERNAME,CHROMOSOME,POSITION,EA,NEA,EAF,BETA,SE,OR,OR_95U,OR_95L,N,NMISS,P
<chr>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
chr1:794332,1,794332,A,G,0.1394230,0.0013,0.03355957,1.0013008,1.069378,0.9375579,31575,21168.41,0.9691
chr1:832359,1,832359,T,C,0.0312500,-0.0932,0.07060045,0.9110113,1.046213,0.7932817,31575,21168.41,0.1868
chr1:834056,1,834056,C,A,0.0288462,-0.0766,0.07745787,0.9262603,1.078119,0.7957920,31575,21168.41,0.3227
chr1:834263,1,834263,T,C,0.0288462,-0.0846,0.07917723,0.9188798,1.073138,0.7867951,31575,21168.41,0.2853
chr1:834956,1,834956,A,G,0.0264423,-0.0704,0.07786061,0.9320209,1.085680,0.8001094,31575,21168.41,0.3659
chr1:838665,1,838665,C,T,0.0288462,-0.0785,0.07129932,0.9245021,1.063161,0.8039271,31575,21168.41,0.2709


In [3]:
df0$MarkerName <- df0$MARKERNAME
df0$StdErr <- df0$SE
df0$Effect <- df0$BETA
df0$`P-value` <- df0$P
df0$CHR <- df0$CHROMOSOME
df0$BP <- df0$POSITION

In [66]:
df0 <- na.omit(df0)

In [76]:
str(df0)

Classes ‘data.table’ and 'data.frame':	5628895 obs. of  20 variables:
 $ MARKERNAME: chr  "chr1:794332" "chr1:832359" "chr1:834056" "chr1:834263" ...
 $ CHROMOSOME: int  1 1 1 1 1 1 1 1 1 1 ...
 $ POSITION  : int  794332 832359 834056 834263 834956 838665 838732 838890 838916 839461 ...
 $ EA        : chr  "A" "T" "C" "T" ...
 $ NEA       : chr  "G" "C" "A" "C" ...
 $ EAF       : num  0.1394 0.0312 0.0288 0.0288 0.0264 ...
 $ BETA      : num  0.0013 -0.0932 -0.0766 -0.0846 -0.0704 ...
 $ SE        : num  0.0336 0.0706 0.0775 0.0792 0.0779 ...
 $ OR        : num  1.001 0.911 0.926 0.919 0.932 ...
 $ OR_95U    : num  1.07 1.05 1.08 1.07 1.09 ...
 $ OR_95L    : num  0.938 0.793 0.796 0.787 0.8 ...
 $ N         : int  31575 31575 31575 31575 31575 31575 31575 31575 31575 31575 ...
 $ NMISS     : num  21168 21168 21168 21168 21168 ...
 $ P         : num  0.969 0.187 0.323 0.285 0.366 ...
 $ MarkerName: chr  "chr1:794332" "chr1:832359" "chr1:834056" "chr1:834263" ...
 $ StdErr    : num  0.03

In [78]:
## EXTRACT CHRS
# Chromosome 1: 155,210,185 -> 155,110,185 - 155,310,185
GBA1_sumstats = subset(df0, CHR==1 & BP > 155110185 & BP< 155310185)
# Chromosome 6: 112,151,452 -> 112,051,452 - 112,251,452
FYN_sumstats = subset(df0, CHR==6 & BP > 111660332 & BP< 111873452)
# Chromosome 12: 40,387,749 -> 40,287,749 - 40,487,749
LRRK2_sumstats = subset(df0, CHR==12 & BP > 40287749 & BP< 40487749)
# Chromosome 4: 90,682,474 -> 90,582,474 - 90,782,474
SNCA_sumstats = subset(df0, CHR==4 & BP > 90582474 & BP< 90782474)
# Chromosome 5: 75,599,208 -> 75,499,208 - 75,699,208
SV2C_sumstats = subset(df0, CHR==5 & BP > 75847464 & BP<76353939)
# Chromosome 7: 70,750,493 -> 70,650,493 - 70,850,493
WBSCR17_sumstats = subset(df0, CHR==7 & BP > 70650493 & BP<70850493)
# Chromosome 3: 182,735,211 -> 182,635,211 - 182,835,211
MCCC1_sumstats = subset(df0, CHR==3 & BP > 182635211 & BP < 182835211)
# Chromosome 1: 226,846,712 -> 226,746,712 - 226,946,712
ITPKB_sumstats = subset(df0, CHR==1 & BP > 226746712 & BP < 226946712)
# Chromosome 4: 77,101,068 -> 77,001,068 - 77,201,068
FAM47E_sumstats = subset(df0, CHR==4 & BP > 77001068 & BP < 77201068)
# Chromosome 11: 83,510,117 -> 83,410,117 - 83,610,117
DLG2_sumstats = subset(df0, CHR==11 & BP > 83410117 & BP < 83610117)
# Chromosome 18: 40,678,235 -> 40,578,235 - 40,778,235
RIT2_sumstats = subset(df0, CHR==18 & BP > 40578235 & BP < 40778235)

In [79]:
## Run for genes
genes <- c("GBA1","FYN","LRRK2","SNCA","SV2C","WBSCR17","MCCC1","ITPKB","FAM47E","DLG2","RIT2")

In [80]:
for (gene in genes) {
    # Assume gene_sumstats is a data frame with summary statistics for each gene
    gene_sumstats <- get(paste0(gene, "_sumstats"))  # Get the data frame for the current gene
    if (is.data.frame(gene_sumstats)) {
        write_tsv(gene_sumstats, paste0("{WORK_DIR}/PAR/", "/", gene, "_variants_eas.tab"))
    } else {
        warning(paste("No data frame found for", gene))
    }
}

In [81]:
## Run for genes
genes <- c("GBA1","FYN","LRRK2","SNCA","SV2C","WBSCR17","MCCC1","ITPKB","FAM47E","DLG2","RIT2")

In [82]:
for (gene in genes) {
    input_file <- paste0("{WORK_DIR}/PAR/", gene, "_variants_eas.tab")
    output_file <- paste0("{WORK_DIR}/PAR/", gene, "_Foo_2020.csv")
    
    # Read in the dataset
    dataset1 <- fread(input_file, header = TRUE, sep = "\t")
    
    # Remove duplicated rows based on the 'MarkerName' column
    dataset1 <- dataset1[!duplicated(dataset1$MarkerName), ]
    
    # Add a new column 'StdErr_squared' by squaring 'StdErr'
    dataset_final <- dataset1 %>% mutate(StdErr_squared = StdErr^2)
    
    # Select the required columns and rename them
    output <- dataset_final[, c("MarkerName", "Effect", "P-value", "StdErr_squared")]
    colnames(output) <- c("SNP", "beta", "P", "varbeta")

    # Remove rows where beta is 0
    output <- output[output$beta != 0, ]
    
    # Write the output to a CSV file
    fwrite(output, file = output_file, na = "NA", quote = FALSE, row.names = FALSE, sep = "\t")
}

In [83]:
## Run for genes
genes <- c("GBA1","FYN","LRRK2","SNCA","SV2C","WBSCR17","MCCC1","ITPKB","FAM47E","DLG2","RIT2")

In [84]:
for (gene in genes) {
    input_file <- paste0("{WORK_DIR}/PAR/", gene, "_Foo_2020.csv")
    output <- fread(input_file, header = TRUE, sep = "\t")
    
    # Check if output has 0 rows
    if (nrow(output) == 0) {
        cat("No rows in output for gene: ", gene, ". Skipping...\n")
        next  # Skip to the next gene in the loop
    }
    
    SNP <- output$SNP
    beta <- output$beta
    varbeta <- output$varbeta
    N <- 31575  # 6724 PD cases vs 31575 total (6724 cases, 24851 controls - Foo et al 2020)
    s <- 0.213
    type <- 'cc'
    
    # Create dataset for fine-mapping
    dataset <- list(
        snp = SNP, 
        beta = beta, 
        varbeta = varbeta, 
        N = N, 
        s = s, 
        type = type)
        
    # Ensure dataset variables are numeric
    dataset$snp <- unlist(dataset$snp)
    dataset$beta <- unlist(dataset$beta)
    dataset$varbeta <- unlist(dataset$varbeta)
    
    # Assuming finemap.abf() works with a list, otherwise convert to a data.frame
    results <- finemap.abf(
        dataset = dataset,
        p1 = 1e-04  # Optional parameter for p-value threshold (can adjust based on your data)
    )
        
    # Check if results has 0 rows
    if (nrow(results) == 0) {
        cat("No results returned for gene: ", gene, ". Skipping...\n")
        next  # Skip to the next gene in the loop
    }
    
    # Combine the results with the original output
    combo <- cbind(results[1:(nrow(results) - 1),], output)
    
    # Subset results where SNP.PP > 0.2
    hits <- subset(combo, SNP.PP > 0.2)
    
    # Save the results to a CSV file
    final_output_file <- paste0("{WORK_DIR}/PAR/", gene, "_results_fine_map_Foo.csv")
    fwrite(combo, file = final_output_file, na = "NA", quote = F, row.names = F, sep = ",")
    }

Warning message in check_dataset(dataset, ""):
“minimum p value is: 1.546e-06
If this is what you expected, this is not a problem.
If this is not as small as you expected, please check you supplied var(beta) and not sd(beta) for the varbeta argument. If that's not the explanation, please check the 02_data vignette.”
Warning message in check_dataset(dataset, ""):
“minimum p value is: 0.007145
If this is what you expected, this is not a problem.
If this is not as small as you expected, please check you supplied var(beta) and not sd(beta) for the varbeta argument. If that's not the explanation, please check the 02_data vignette.”
Warning message in check_dataset(dataset, ""):
“minimum p value is: 0.004287
If this is what you expected, this is not a problem.
If this is not as small as you expected, please check you supplied var(beta) and not sd(beta) for the varbeta argument. If that's not the explanation, please check the 02_data vignette.”


In [85]:
# Define the directory containing the CSV files
input_directory <- "{WORK_DIR}/PAR/"  # Replace with your directory path

# List all files with the pattern "results_fine_map.csv" in the directory
file_list <- list.files(input_directory, pattern = "_results_fine_map_Foo.csv$", full.names = TRUE)

# Initialize an empty list to store the results
results_list <- list()

# Loop through each file
for (file in file_list) {
  # Extract the gene name from the file name (remove the "_results_fine_map.csv" suffix)
  gene <- gsub("_results_fine_map_Foo\\.csv$", "", basename(file))
  
  # Read the CSV file
  data <- read.csv(file)
  
  # Select the SNP with the highest SNP.PP value
  best_snp <- data %>%
    slice_max(SNP.PP, n = 1) %>%  # Select row(s) with the max SNP.PP
    mutate(gene = gene)           # Add the gene name
  
  # Append to the results list
  results_list[[gene]] <- best_snp
}

# Combine all results into a single dataframe
final_results <- bind_rows(results_list)

# Export the results to a CSV file
output_file <- "top_snp_per_gene_eas.csv"  # Desired output file name
write.csv(final_results, output_file, row.names = FALSE)

# Print the first few rows of the final results
print(head(final_results))

            V.        z.        r.     lABF.            snp prior       SNP.PP
1 0.0005231275 -5.626972 0.9870906 13.452134 chr11:83510117 1e-04 0.1628960866
2 0.0005692900 -5.658053 0.9859675 13.648977  chr4:77101068 1e-04 0.2575197109
3 0.0005159110 -2.690009 0.9872665  1.390247 chr6:111871349 1e-04 0.0004015336
4 0.0042537873  4.805196 0.9038774  9.264158 chr1:155205203 1e-04 0.3488126265
5 0.0019876177  6.141395 0.9526618 16.440425 chr1:226885608 1e-04 0.0544968760
6 0.0030640578 -8.125891 0.9288488 29.344516 chr12:40458384 1e-04 0.5307933273
             SNP    beta         P      varbeta   gene
1 chr11:83510117 -0.1287 1.830e-08 0.0005231275   DLG2
2  chr4:77101068 -0.1350 1.530e-08 0.0005692900 FAM47E
3 chr6:111871349 -0.0611 7.145e-03 0.0005159110    FYN
4 chr1:155205203  0.3134 1.550e-06 0.0042537873   GBA1
5 chr1:226885608  0.2738 8.180e-10 0.0019876177  ITPKB
6 chr12:40458384 -0.4498 5.140e-16 0.0030640578  LRRK2


In [86]:
final_results

V.,z.,r.,lABF.,snp,prior,SNP.PP,SNP,beta,P,varbeta,gene
<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
0.0005231275,-5.626972,0.9870906,13.452134,chr11:83510117,1e-04,0.1628960866,chr11:83510117,-0.1287,1.830e-08,0.0005231275,DLG2
0.0005692900,-5.658053,0.9859675,13.648977,chr4:77101068,1e-04,0.2575197109,chr4:77101068,-0.1350,1.530e-08,0.0005692900,FAM47E
0.0005159110,-2.690009,0.9872665,1.390247,chr6:111871349,1e-04,0.0004015336,chr6:111871349,-0.0611,7.145e-03,0.0005159110,FYN
0.0042537873,4.805196,0.9038774,9.264158,chr1:155205203,1e-04,0.3488126265,chr1:155205203,0.3134,1.550e-06,0.0042537873,GBA1
0.0019876177,6.141395,0.9526618,16.440425,chr1:226885608,1e-04,0.0544968760,chr1:226885608,0.2738,8.180e-10,0.0019876177,ITPKB
0.0030640578,-8.125891,0.9288488,29.344516,chr12:40458384,1e-04,0.5307933273,chr12:40458384,-0.4498,5.140e-16,0.0030640578,LRRK2
0.0005327549,-8.292361,0.9868562,31.763819,chr3:182731890,1e-04,0.1530013394,chr3:182731890,-0.1914,8.610e-17,0.0005327549,MCCC1
0.0005413375,5.449858,0.9866473,12.494166,chr18:40678235,1e-04,0.1109885881,chr18:40678235,0.1268,5.040e-08,0.0005413375,RIT2
0.0005694186,8.209536,0.9859643,31.092188,chr4:90709236,1e-04,0.2869566535,chr4:90709236,0.1959,2.980e-16,0.0005694186,SNCA
